### Prepare Model

In [2]:
import numpy as np
import torch
import tensorflow as tf

device = 'cuda' if torch.cuda.is_available() else 'cpu'

2024-01-30 23:58:38.039221: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-30 23:58:38.072168: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 23:58:38.072197: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 23:58:38.072918: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 23:58:38.077703: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# Load TensorFlow MNIST data
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize and flatten the images
train_images_tf = train_images.reshape((-1, 28*28)) / 255.0
test_images_tf = test_images.reshape((-1, 28*28)) / 255.0

In [4]:
from tensorflow import keras
num_classes = 10

model_tf = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28*28,)),  # Adjusted for 28x28 images
#    keras.layers.Dense(128, activation='relu'),     # Increased number of neurons
    keras.layers.Dense(56, activation='relu'),      # Additional hidden layer
    keras.layers.Dense(num_classes, activation='softmax')  # Output layer for 10 classes
])

model_tf.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

2024-01-30 23:58:50.860708: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-30 23:58:50.861750: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-30 23:58:50.861863: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [5]:
# Train the model
model_tf.fit(train_images_tf, train_labels, epochs=5, batch_size=32, validation_split=0.1)

# Evaluate the model
test_loss, test_acc = model_tf.evaluate(test_images_tf, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/5


2024-01-30 23:58:57.556004: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-01-30 23:58:57.597950: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f3270003070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-30 23:58:57.597968: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-01-30 23:58:57.602834: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-30 23:58:57.615763: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
I0000 00:00:1706630337.675971  398306 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1688/1688 [==============================] - 2s 1ms/step - loss: 0.3203 - accuracy: 0.9100 - val_loss: 0.1511 - val_accuracy: 0.9580
Epoch 2/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1595 - accuracy: 0.9542 - val_loss: 0.1119 - val_accuracy: 0.9673
Epoch 3/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.1159 - accuracy: 0.9664 - val_loss: 0.1066 - val_accuracy: 0.9670
Epoch 4/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0918 - accuracy: 0.9723 - val_loss: 0.0940 - val_accuracy: 0.9727
Epoch 5/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0758 - accuracy: 0.9768 - val_loss: 0.0907 - val_accuracy: 0.9747
313/313 - 0s - loss: 0.0918 - accuracy: 0.9722 - 229ms/epoch - 731us/step

Test accuracy: 0.9721999764442444


In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_tf)
tflite_model = converter.convert()

with open('model_tf.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpvk4n6cbj/assets


INFO:tensorflow:Assets written to: /tmp/tmpvk4n6cbj/assets
2024-01-31 00:29:59.152855: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-01-31 00:29:59.152873: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-01-31 00:29:59.153013: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpvk4n6cbj
2024-01-31 00:29:59.153472: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-01-31 00:29:59.153481: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpvk4n6cbj
2024-01-31 00:29:59.154789: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2024-01-31 00:29:59.180267: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpvk4n6cbj
2024-01-31 00:29:59.186787: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [6]:
import tensorflow_model_optimization as tfmot

# Apply quantization to the layers
quantize_model = tfmot.quantization.keras.quantize_model

q_aware_model = quantize_model(model_tf)

# 'quantize_model' requires a recompile
q_aware_model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

q_aware_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLa  (None, 784)               3         
 yer)                                                            
                                                                 
 quant_dense (QuantizeWrapp  (None, 56)                43965     
 erV2)                                                           
                                                                 
 quant_dense_1 (QuantizeWra  (None, 10)                575       
 pperV2)                                                         
                                                                 
Total params: 44543 (174.00 KB)
Trainable params: 44530 (173.95 KB)
Non-trainable params: 13 (52.00 Byte)
_________________________________________________________________


In [7]:
# Train the model
q_aware_model.fit(train_images_tf, train_labels, epochs=5, batch_size=32, validation_split=0.1)

# Evaluate the model
test_loss, test_acc = q_aware_model.evaluate(test_images_tf, test_labels, verbose=2)
print('\nTest accuracy:', test_acc)

Epoch 1/5
1688/1688 [==============================] - 3s 1ms/step - loss: 0.0656 - accuracy: 0.9801 - val_loss: 0.0917 - val_accuracy: 0.9742
Epoch 2/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0544 - accuracy: 0.9836 - val_loss: 0.0998 - val_accuracy: 0.9743
Epoch 3/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0474 - accuracy: 0.9850 - val_loss: 0.0880 - val_accuracy: 0.9758
Epoch 4/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0406 - accuracy: 0.9875 - val_loss: 0.0907 - val_accuracy: 0.9767
Epoch 5/5
1688/1688 [==============================] - 2s 1ms/step - loss: 0.0361 - accuracy: 0.9884 - val_loss: 0.0969 - val_accuracy: 0.9763
313/313 - 0s - loss: 0.1000 - accuracy: 0.9713 - 254ms/epoch - 811us/step

Test accuracy: 0.9713000059127808


In [8]:
import tensorflow as tf

# Create a converter
converter = tf.lite.TFLiteConverter.from_keras_model(q_aware_model)

# Indicate that you want to perform default optimizations,
# which include quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Define a generator function that provides your test data's numpy arrays
def representative_data_gen():
  for i in range(500):
    yield [np.array(train_images[i:i+1], dtype=np.float32)]

# Use the generator function to guide the quantization process
converter.representative_dataset = representative_data_gen

# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Set the input and output tensors to int8
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Convert the model
tflite_model = converter.convert()

# Save the model to disk
open("q_aware_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp_hiyp0no/assets


INFO:tensorflow:Assets written to: /tmp/tmp_hiyp0no/assets
/home/guy1m0/Desktop/ZKML-Benchmark/env/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2024-01-31 00:03:37.720070: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2024-01-31 00:03:37.720084: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2024-01-31 00:03:37.720262: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp_hiyp0no
2024-01-31 00:03:37.721044: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-01-31 00:03:37.721054: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp_hiyp0no
2024-01-31 00:03:37.723067: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 op

46896

### Convert model to tflite and msgpack

In [17]:
convert_path = "./tools/converter.py"
model_in_path = "model_tf.tflite"
model_out_path = "converted_model.msgpack"
config_out_path = "config.msgpack"

scale_factor = 512
k = 17
num_cols = 10 # not understand
num_randoms = 1024 # not understand

command_1 = ["python", f"{convert_path}", "--model", f"{model_in_path}",
           "--model_output", f"{model_out_path}", "--config_output",
           f"{config_out_path}", "--scale_factor", str(scale_factor),
           "--k", str(k), "--num_cols", str(num_cols), "--num_randoms",
           str(num_randoms)]



In [18]:
import subprocess, re
# Start the subprocess
subprocess.run(command_1)

2024-01-31 00:30:12.759571: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-31 00:30:12.779368: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-31 00:30:12.779391: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-31 00:30:12.779905: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-31 00:30:12.782972: I tensorflow/core/platform/cpu_feature_guar

[{'layer_type': 'FullyConnected', 'inp_idxes': [0, 3, 2], 'inp_shapes': [[1, 784], [56, 784], [56]], 'out_idxes': [5], 'out_shapes': [[1, 56]], 'params': [1], 'mask': []}, {'layer_type': 'FullyConnected', 'inp_idxes': [5, 4, 1], 'inp_shapes': [[1, 56], [10, 56], [10]], 'out_idxes': [6], 'out_shapes': [[1, 10]], 'params': [0], 'mask': []}, {'layer_type': 'Softmax', 'inp_idxes': [6], 'inp_shapes': [[1, 10]], 'out_idxes': [7], 'out_shapes': [[1, 10]], 'params': [], 'mask': []}]

keep tensors: {0, 1, 2, 3, 4, 5, 6}
skipping generated tensor: 0, b'serving_default_input_1:0'
skipping generated tensor: 5, b'sequential/dense/MatMul;sequential/dense/Relu;sequential/dense/BiasAdd'
skipping generated tensor: 6, b'sequential/dense_1/MatMul;sequential/dense_1/BiasAdd'

{'layer_type': 'Softmax', 'inp_idxes': [6], 'inp_shapes': [[1, 10]], 'out_idxes': [7], 'out_shapes': [[1, 10]], 'params': [], 'mask': []}
dict_keys(['global_sf', 'k', 'num_cols', 'num_random', 'inp_idxes', 'out_idxes', 'layers', 'ten

CompletedProcess(args=['python', './tools/converter.py', '--model', 'model_tf.tflite', '--model_output', 'converted_model.msgpack', '--config_output', 'config.msgpack', '--scale_factor', '512', '--k', '17', '--num_cols', '10', '--num_randoms', '1024'], returncode=0)

### Convert img to npy format

In [19]:
index = 0
x = test_images_tf[index]

print(x.dtype, x.shape)
np.save(str(index)+'.npy', x)


float64 (784,)


In [22]:
convert_path = "./tools/input_converter.py"
model_in_path = "model_tf.tflite"
model_out_path = "converted_model.msgpack"
config_out_path = "config.msgpack"
input_path = str(index)+ ".npy"
output_path = "img_" + str(index) + ".msgpack"

scale_factor = 512
k = 17
num_cols = 10 # not understand
num_randoms = 1024 # not understand

command_2 = ["python", f"{convert_path}", "--model_config", f"{config_out_path}",
           "--inputs", input_path, "--output", output_path]


In [23]:
subprocess.run(command_2)

CompletedProcess(args=['python', './tools/input_converter.py', '--model_config', 'config.msgpack', '--inputs', '0.npy', '--output', 'img_0.msgpack'], returncode=0)

### Test, Compile and Verify

In [24]:
test = "./bin/test_circuit"
model_out_path = "converted_model.msgpack"
img_out_path = "img_" + str(index) + ".msgpack"

command_3 = [test, model_out_path, img_out_path]

subprocess.run(command_3)


Processing layer 0, type: FullyConnected, inp_idxes: [0, 3, 2], out_idxes: [5], layer_params: [1]
Out 0 shape: [1, 56]

Processing layer 1, type: FullyConnected, inp_idxes: [5, 4, 1], out_idxes: [6], layer_params: [0]
Out 0 shape: [1, 10]

Processing layer 2, type: Softmax, inp_idxes: [6], out_idxes: [7], layer_params: []
Out 0 shape: [1, 10]

final out[0] x: 0 (0)
final out[1] x: 0 (0)
final out[2] x: 0 (0)
final out[3] x: 0 (0)
final out[4] x: 0 (0)
final out[5] x: 0 (0)
final out[6] x: 0 (0)
final out[7] x: 512 (1)
final out[8] x: 0 (0)
final out[9] x: 0 (0)
final out idxes: [7]
Processing layer 0, type: FullyConnected, inp_idxes: [0, 3, 2], out_idxes: [5], layer_params: [1]
Out 0 shape: [1, 56]

Processing layer 1, type: FullyConnected, inp_idxes: [5, 4, 1], out_idxes: [6], layer_params: [0]
Out 0 shape: [1, 10]

Processing layer 2, type: Softmax, inp_idxes: [6], out_idxes: [7], layer_params: []
Out 0 shape: [1, 10]

final out[0] x: 0 (0)
final out[1] x: 0 (0)
final out[2] x: 0 (0)

CompletedProcess(args=['./bin/test_circuit', 'converted_model.msgpack', 'img_0.msgpack'], returncode=0)

In [27]:
time_circuit = "./bin/time_circuit"
model_out_path = "converted_model.msgpack"
img_out_path = "img_" + str(index) + ".msgpack"

command_4 = [time_circuit, model_out_path, img_out_path, "kzg"]

subprocess.run(command_4)

Time elapsed in params construction: 3.973671619s
Processing layer 0, type: FullyConnected, inp_idxes: [0, 3, 2], out_idxes: [5], layer_params: [1]
Out 0 shape: [1, 56]

Processing layer 1, type: FullyConnected, inp_idxes: [5, 4, 1], out_idxes: [6], layer_params: [0]
Out 0 shape: [1, 10]

Processing layer 2, type: Softmax, inp_idxes: [6], out_idxes: [7], layer_params: []
Out 0 shape: [1, 10]

final out[0] x: 0 (0)
final out[1] x: 0 (0)
final out[2] x: 0 (0)
final out[3] x: 0 (0)
final out[4] x: 0 (0)
final out[5] x: 0 (0)
final out[6] x: 0 (0)
final out[7] x: 0 (0)
final out[8] x: 0 (0)
final out[9] x: 0 (0)
final out idxes: [7]
Time elapsed in generating vkey: 1.831431293s
vkey size: 148936 bytes
Processing layer 0, type: FullyConnected, inp_idxes: [0, 3, 2], out_idxes: [5], layer_params: [1]
Out 0 shape: [1, 56]

Processing layer 1, type: FullyConnected, inp_idxes: [5, 4, 1], out_idxes: [6], layer_params: [0]
Out 0 shape: [1, 10]

Processing layer 2, type: Softmax, inp_idxes: [6], ou

CompletedProcess(args=['./bin/time_circuit', 'converted_model.msgpack', 'img_0.msgpack', 'kzg'], returncode=0)